<a href="https://colab.research.google.com/github/badboycherry/crops-deepLearning/blob/main/%EA%B6%8C%EB%AF%BC%EA%B2%BD_Resnet_pepper_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models


# 각 카테고리에 대한 이미지 갯수 확인 함수
def count_images(directory):
    try:
        # 디렉토리 안의 파일 목록을 가져옵니다.
        files = os.listdir(directory)

        # leaf와 mainBody 폴더를 확인하고, 각 폴더의 이미지 갯수를 출력합니다.
        for category in ['leaf', 'mainBody']:
            category_path = os.path.join(directory, category)
            image_count = 0

            # leaf 또는 mainBody 폴더 안의 각 파일에 대해 반복합니다.
            for file in os.listdir(category_path):
                _, file_extension = os.path.splitext(file)
                image_extensions = ['.jpg', '.png']

                # 만약 파일의 확장자가 이미지 확장자 중 하나라면 image_count를 증가시킵니다.
                if file_extension.lower() in image_extensions:
                    image_count += 1

            # 결과를 출력합니다.
            print(f"디렉토리 '{category_path}'에 있는 이미지 파일의 수: {image_count}")

    except Exception as e:
        print(f"에러 발생: {e}")

# 각 디렉토리에 대해 이미지 파일의 수를 출력합니다.
count_images("/content/drive/MyDrive/1조 공유/pepper_data/pepper_disease")
count_images("/content/drive/MyDrive/1조 공유/pepper_data/pepper_normal")


디렉토리 '/content/drive/MyDrive/1조 공유/pepper_data/pepper_disease/leaf'에 있는 이미지 파일의 수: 1141
디렉토리 '/content/drive/MyDrive/1조 공유/pepper_data/pepper_disease/mainBody'에 있는 이미지 파일의 수: 609
디렉토리 '/content/drive/MyDrive/1조 공유/pepper_data/pepper_normal/leaf'에 있는 이미지 파일의 수: 2683
디렉토리 '/content/drive/MyDrive/1조 공유/pepper_data/pepper_normal/mainBody'에 있는 이미지 파일의 수: 317


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 로드 및 전처리
batch_size = 32
image_size = (224, 224)

# 데이터 증강 설정
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # 80%는 훈련 데이터, 20%는 검증 데이터로 분할

# 훈련 데이터 생성
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/1조 공유/pepper_data/',
                                                    target_size=image_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training')

# 검증 데이터 생성
validation_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/1조 공유/pepper_data/',
                                                         target_size=image_size,
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation')

Found 3800 images belonging to 2 classes.
Found 950 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# ResNet 모델 불러오기 (사전 훈련된 가중치를 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(2, activation='softmax'))  # 클래스 수에 맞게 조절

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
# 모델 훈련
epochs = 5

model.fit(train_generator,
          epochs=epochs,
          validation_data=validation_generator)

Epoch 1/5
119/119 [==============================] - 5304s 44s/step - loss: 0.0739 - accuracy: 0.9761 - val_loss: 3529.4211 - val_accuracy: 0.6316
Epoch 2/5
 77/119 [==================>...........] - ETA: 15:29 - loss: 0.0668 - accuracy: 0.9825

KeyboardInterrupt: ignored

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import io

# UnidentifiedImageError를 임포트합니다.
from PIL import UnidentifiedImageError


# ImageDataGenerator를 사용하여 이미지를 읽어오는 코드
for x_batch, y_batch in train_generator:
    try:
        # 이미지를 디버깅하기 위해 첫 번째 배치의 이미지를 확인
        error_image_bytes = x_batch[0]
        error_image = Image.open(io.BytesIO(error_image_bytes))

        # 이미지를 시각화
        plt.imshow(error_image)
        plt.show()

        # 모델에 입력으로 전달하는 코드 등을 여기에 추가
        # model.fit() 등...

    except UnidentifiedImageError as e:
        # 이미지 식별 오류가 발생한 경우 예외 처리
        print(f"UnidentifiedImageError: {e}")
        print("Skipping this batch and continuing training.")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f42582c0>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f42582c0>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f42582c0>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f461ccc0>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f461d440>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f461d2b0>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7905f4258360>
Skipping this batch and continuing training.
UnidentifiedImageError: cannot identify image file <_io

UnidentifiedImageError: ignored

In [ ]:
# 새로운 이미지를 예측하는 부분
img_path = ''  # 예측할 이미지 경로를 적절히 수정하세요
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # 이미지를 [0, 1] 범위로 정규화


In [ ]:
# 모델에 이미지 주입하여 예측 수행
predictions = model.predict(img_array)

In [ ]:
# 예측 결과 출력
class_labels = ['pepper_disease/leaf', 'pepper_disease/mainBody', 'pepper_normal/leaf', 'pepper_normal/mainBody']
predicted_class = class_labels[np.argmax(predictions)]
print(f"The predicted class is: {predicted_class}")